### Install project

In [1]:
!git clone https://github.com/kornelro/cv_project.git

fatal: destination path 'cv_project' already exists and is not an empty directory.


In [2]:
!pip install cv_project/.

Processing ./cv_project
  Created wheel for src: filename=src-0.1.0-cp36-none-any.whl size=16502 sha256=89717b2db194b5bca98ecded0aec15ea9d78577713651b4089a0f6e78b096f75
  Stored in directory: /tmp/pip-ephem-wheel-cache-klvqtit7/wheels/fc/1b/40/72379bece2bafba26136079aeac87421c9221abafeb9c66746
Successfully built src
  Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0


### Mount drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
DRIVE_PATH = '/content/drive/Shareddrives/CV-PROJECT/'

### Imports

In [5]:
import pickle
import numpy as np
import tensorflow.keras as K
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from src.data.make_dataset import make_dataset
from src.data.image_loaders import AerialCarsFixedSizeImageLoader
from src.data.dataset_loaders import AerialCarsDatasetLoader
from src.features.pipelines import RawImageToImage
from src.features.processors import NormImage
from src.models.classifiers import ResnetModelWrapper
from src.models.detectors import SliderProbDetector
from src.data.sliders import SlidingWindow
from src.evaluation import validate_model, precision, recall, f1_score

# Aerial-Cars-Dataset
# ResNet50 1 epoch
# Pipeline: Norm

### Load dataset

In [6]:
DATA_PATH = DRIVE_PATH+'data/processed/aerial-cars-dataset/resnet/data_resnet.pickle'

In [7]:
with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

### Train ResNet50

In [8]:
TEST_SIZE = 0.1
BATCH_SIZE = 100
EPOCHS = 1

In [9]:
X = np.array(list(map(lambda x: x[1], data)))
y = np.array(list(map(lambda x: x[2], data)))
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=42
  )


input_t = K.Input(shape=(X[0].shape))
res_model = K.applications.ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_tensor=input_t
)

for layer in res_model.layers:
    layer.trainable = False

model = K.models.Sequential()
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.Dense(2, activation='softmax'))


model.compile(
    optimizer=K.optimizers.Adam(0.001),
    loss=K.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = model.fit(X_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS, 
            validation_data=(X_test, y_test)
)

43/43 [==============================] - 13s 91ms/step - loss: 0.2650 - accuracy: 0.9210 - val_loss: 0.0452 - val_accuracy: 0.9852


### Validation

In [10]:
del data

In [11]:
BND_BOX_SIZE = (80, 80)
STEP_SIZE = 20

dataset_loader = AerialCarsDatasetLoader(
    image_loader=AerialCarsFixedSizeImageLoader(
        bnd_box_size=BND_BOX_SIZE
    )
)

process_pipeline=RawImageToImage(
    processors=[
        NormImage()
    ]
)

sliding_window = SlidingWindow(
    step_size=STEP_SIZE,
    window_size=BND_BOX_SIZE
)

treshold = 0.7; nms_overlap=0.4


In [13]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.7,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/aerial-cars-dataset/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/aerial-cars-dataset/norm/t7nms4_1e',
    images_files_types=('jpg', 'JPG', 'png'),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

 11%|█         | 1/9 [00:06<00:51,  6.47s/it]

201


 22%|██▏       | 2/9 [00:11<00:41,  5.96s/it]

219


 33%|███▎      | 3/9 [00:15<00:33,  5.56s/it]

...

 78%|███████▊  | 7/9 [00:56<00:18,  9.46s/it]

225
204


100%|██████████| 9/9 [01:06<00:00,  7.34s/it]

143



Precision: 0.259
Recall: 0.867
F1Score: 0.398


treshold = 0.8; nms_overlap=0.4

In [14]:
del processed_images

In [15]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.8,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/aerial-cars-dataset/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/aerial-cars-dataset/norm/t8nms4_1e',
    images_files_types=('jpg', 'JPG', 'png'),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

 11%|█         | 1/9 [00:04<00:38,  4.87s/it]

193


 22%|██▏       | 2/9 [00:09<00:34,  4.88s/it]

208


 33%|███▎      | 3/9 [00:14<00:29,  4.87s/it]

...

 89%|████████▉ | 8/9 [00:59<00:07,  7.97s/it]

198


100%|██████████| 9/9 [01:04<00:00,  7.20s/it]

135



Precision: 0.287
Recall: 0.894
F1Score: 0.434


treshold = 0.9; nms_overlap=0.4

In [16]:
del processed_images

In [12]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/aerial-cars-dataset/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/aerial-cars-dataset/norm/t9nms4_1e',
    images_files_types=('jpg', 'JPG', 'png'),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

 11%|█         | 1/9 [00:06<00:50,  6.35s/it]

203


 22%|██▏       | 2/9 [00:10<00:40,  5.84s/it]

238


 33%|███▎      | 3/9 [00:15<00:32,  5.48s/it]

...

 89%|████████▉ | 8/9 [01:01<00:08,  8.15s/it]

207


100%|██████████| 9/9 [01:06<00:00,  7.37s/it]

157



Precision: 0.257
Recall: 0.903
F1Score: 0.4


treshold = 0.95; nms_overlap=0.4

In [ ]:
del processed_images

In [13]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.95,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/aerial-cars-dataset/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/aerial-cars-dataset/norm/t95nms4_1e',
    images_files_types=('jpg', 'JPG', 'png'),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

 11%|█         | 1/9 [00:04<00:39,  4.93s/it]

188


 22%|██▏       | 2/9 [00:09<00:34,  4.91s/it]

226


 33%|███▎      | 3/9 [00:14<00:29,  4.90s/it]

...

 89%|████████▉ | 8/9 [00:59<00:07,  7.83s/it]

193


100%|██████████| 9/9 [01:03<00:00,  7.10s/it]

151



Precision: 0.259
Recall: 0.85
F1Score: 0.397


treshold = 0.9; nms_overlap=0.3

In [ ]:
processed_images = None

slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.3
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/aerial-cars-dataset/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/aerial-cars-dataset/norm/t9nms3',
    images_files_types=('jpg', 'JPG', 'png'),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

 11%|█         | 1/9 [00:06<00:52,  6.55s/it]

183


 22%|██▏       | 2/9 [00:11<00:41,  5.98s/it]

217


 33%|███▎      | 3/9 [00:15<00:33,  5.60s/it]

...

 89%|████████▉ | 8/9 [01:01<00:08,  8.10s/it]

188


100%|██████████| 9/9 [01:06<00:00,  7.35s/it]

148



Precision: 0.287
Recall: 0.69
F1Score: 0.405


treshold = 0.9; nms_overlap=0.5

In [ ]:
processed_images = None

slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.5
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/aerial-cars-dataset/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/aerial-cars-dataset/norm/t9nms5',
    images_files_types=('jpg', 'JPG', 'png'),
    annotations_files_types=('txt',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

 11%|█         | 1/9 [00:06<00:49,  6.19s/it]

188


 22%|██▏       | 2/9 [00:10<00:39,  5.71s/it]

253


 33%|███▎      | 3/9 [00:15<00:32,  5.38s/it]

...

 89%|████████▉ | 8/9 [01:00<00:08,  8.06s/it]

187


100%|██████████| 9/9 [01:05<00:00,  7.28s/it]

159



Precision: 0.257
Recall: 0.867
F1Score: 0.396
